# CS-433 Project 1: ML Higgs 
Changling Li, Julian Blackwell, Luca Bataillard

In [48]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [49]:
from proj1_helpers import *
from implementations import *

In [50]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tx, ids = load_csv_data(DATA_TRAIN_PATH)

## Exploratory data analysis

### Step 1: Inspecting the labels

We begin our data analysis by inspecting the `y` array of labels. According to the documentation, all labels are $-1$ for background 'b' and $1$ is for signal 's'. We check this below:

In [51]:
((y == 1) | (y == -1)).all()

True

### Step 2: Inspecting the features

We first read appendix B of the original Higgs paper and discover several key points:
* All variables are floating point and continuous, apart from `PRI_jet_num`, which ranges in $\{0, 1, 2, 3\}$
* Undefined values are denoted by the value $-999.0$

We inspect the features of `tx` to confirm these claims. We can see that `PRI_jet_num` corresponds to column 22. We decide to do no further processing to this variable and treat it as continuous. We notice several undefinded variables.

In [52]:
N, D = tx.shape
print(f"Number of rows: {N}")
print(f"Number of columns: {D}")

Number of rows: 250000
Number of columns: 30


In [53]:
for i in range(D):
    col = tx[:5, i]
    print(f"Column {i:2d}: {col}")

Column  0: [ 138.47   160.937 -999.     143.905  175.864]
Column  1: [ 51.655  68.768 162.172  81.417  16.915]
Column  2: [ 97.827 103.235 125.953  80.943 134.805]
Column  3: [27.98  48.146 35.635  0.414 16.405]
Column  4: [ 9.10e-01 -9.99e+02 -9.99e+02 -9.99e+02 -9.99e+02]
Column  5: [ 124.711 -999.    -999.    -999.    -999.   ]
Column  6: [   2.666 -999.    -999.    -999.    -999.   ]
Column  7: [3.064 3.473 3.148 3.31  3.891]
Column  8: [41.928  2.078  9.336  0.414 16.405]
Column  9: [197.76  125.157 197.814  75.968  57.983]
Column 10: [1.582 0.879 3.776 2.354 1.056]
Column 11: [ 1.396  1.414  1.414 -1.285 -1.385]
Column 12: [ 2.00e-01 -9.99e+02 -9.99e+02 -9.99e+02 -9.99e+02]
Column 13: [32.638 42.014 32.154 22.647 28.209]
Column 14: [ 1.017  2.039 -0.705 -1.655 -2.197]
Column 15: [ 0.381 -3.011 -2.093  0.01  -2.231]
Column 16: [ 51.626  36.918 121.409  53.321  29.774]
Column 17: [ 2.273  0.501 -0.953 -0.522  0.798]
Column 18: [-2.414  0.103  1.052 -3.1    1.569]
Column 19: [16.824

### Step 3: Summary statistics

By computing the summary statistics on the data, we notice that undefined values are not present in all columns. However, in columns where they are present, they comprise a large proportion of the data. There are too many such rows to be simply discarded as outliers, so we will treat them as regular datapoints for the time being.

We also notice that each feature follows a very different distribution. To ease model computational cost and improve model performance, we will scale the dataset by subtracting the mean and dividing by the standard deviation. 

In [54]:
display_summary_statistics(tx)

Column |   Mean   |  Median  | Std dev  |   Max    |    Min   | # Undefined | % Undefined 
     0 |  -49.023    406.345    105.012   1192.026   -999.000    38114.000        15.246
     1 |   49.240     35.345     46.524    690.075      0.000        0.000         0.000
     2 |   81.182     40.829     73.752   1349.351      6.329        0.000         0.000
     3 |   57.896     63.656     38.468   2834.999      0.000        0.000         0.000
     4 | -708.421    454.480   -999.000      8.503   -999.000   177457.000        70.983
     5 | -601.237    657.971   -999.000   4974.979   -999.000   177457.000        70.983
     6 | -709.357    453.019   -999.000     16.690   -999.000   177457.000        70.983
     7 |    2.373      0.783      2.492      5.684      0.208        0.000         0.000
     8 |   18.917     22.273     12.316   2834.999      0.000        0.000         0.000
     9 |  158.432    115.706    120.665   1852.462     46.104        0.000         0.000
    10 |    1.438  

## Feature processing

In [55]:
tx = standardize(tx)

In [47]:
initial_w = np.zeros(D)
max_iters = 20
gamma = 0.001
seed = 30

y[y < 0] = 0

w, loss = logistic_regression(y, tx, initial_w, max_iters, gamma, verbose=False)
w, loss

[1. 0. 0. ... 1. 0. 0.] [[ 0.46141372  0.06833197  0.40768027 ...  1.5668      1.55858439
   0.4125105 ]
 [ 0.51670419  0.55250482  0.54013641 ... -0.63936657 -0.63936694
  -0.27381996]
 [-2.33785898  3.19515553  1.09655998 ... -0.63936657 -0.63936694
  -0.29396985]
 ...
 [ 0.38016991  0.31931645 -0.13086367 ... -0.63936657 -0.63936694
  -0.31701723]
 [ 0.35431502 -0.84532397 -0.30297338 ... -0.63936657 -0.63936694
  -0.74543941]
 [-2.33785898  0.66533608 -0.25352276 ... -0.63936657 -0.63936694
  -0.74543941]] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[1. 0. 0. ... 1. 0. 0.] [[ 0.46141372  0.06833197  0.40768027 ...  1.5668      1.55858439
   0.4125105 ]
 [ 0.51670419  0.55250482  0.54013641 ... -0.63936657 -0.63936694
  -0.27381996]
 [-2.33785898  3.19515553  1.09655998 ... -0.63936657 -0.63936694
  -0.29396985]
 ...
 [ 0.38016991  0.31931645 -0.13086367 ... -0.63936657 -0.63936694
  -0.31701723]
 [ 0.35431502 -0.84532397 -0.30297338 

/home/luca/Coding/EPFL/CS-433_ML/ML-Project-1/src/implementations.py:31: RuntimeWarning: divide by zero encountered in log
  lhs = y.T.dot()


[1. 0. 0. ... 1. 0. 0.] [[ 0.46141372  0.06833197  0.40768027 ...  1.5668      1.55858439
   0.4125105 ]
 [ 0.51670419  0.55250482  0.54013641 ... -0.63936657 -0.63936694
  -0.27381996]
 [-2.33785898  3.19515553  1.09655998 ... -0.63936657 -0.63936694
  -0.29396985]
 ...
 [ 0.38016991  0.31931645 -0.13086367 ... -0.63936657 -0.63936694
  -0.31701723]
 [ 0.35431502 -0.84532397 -0.30297338 ... -0.63936657 -0.63936694
  -0.74543941]
 [-2.33785898  0.66533608 -0.25352276 ... -0.63936657 -0.63936694
  -0.74543941]] [  35.27780255 -105.84785402  -27.42413587   70.36302115   58.2971541
   99.46121434   57.47259349    2.94703164  -14.83650994   49.77768928
  -47.38701357   80.19558463   58.07093127   77.47990776   -0.5919553
   -0.82913987   20.68350723    0.4107046     0.98773136   38.54645119
    1.51507854   34.63471113   34.73069883   39.69630375   37.14515905
   37.14458714   57.75631862   58.00914509   58.00182577   36.38926872]
[1. 0. 0. ... 1. 0. 0.] [[ 0.46141372  0.06833197  0.407680

(array([ 22.20312652, -77.76404735, -55.62000778,  15.44948817,
        -35.72999804,  78.86199309, -37.8113595 ,  85.42541824,
        -44.78236039, -30.45680853, -35.56773279,   9.32999669,
        -36.29719691,  86.22525133,  -1.19203601,  -0.95320447,
         58.30945253,  -0.56095583,   0.86884652,  25.86351231,
         -0.51399815, -51.70265894, -80.51608606, -32.47376776,
        -30.45558266, -30.45287621, -36.74364419, -36.45584317,
        -36.47472262, -68.79632601]), nan)

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)